#  Export MNIST CNN to ONNX (Opset 11)


This notebook defines a simple CNN model in PyTorch for MNIST digit classification,
exports it to ONNX format with `opset_version=11`, and saves it as `mnist_opset11.onnx`.


In [ ]:

#  Install necessary libraries
!pip install torch torchvision onnx -q


In [ ]:

import torch
import torch.nn as nn
import onnx
import numpy as np


##  Define a Simple CNN Model

In [ ]:

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        return self.net(x)


##  Instantiate and Prepare the Model

In [ ]:

model = MNISTModel()
model.eval()


##  Export the Model to ONNX

In [ ]:

dummy_input = torch.randn(1, 1, 28, 28)

onnx_path = "mnist_opset11.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    opset_version=11,
    dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}}
)
print(" Exported model to", onnx_path)


##  Validate the ONNX File

In [ ]:

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print(" ONNX model is valid!")
